In [1]:
#note: temp readings are 20s apart

import pandas as pd
import os
import matplotlib.pyplot as plt
import random
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
import numpy as np

os.chdir('data')

In [2]:
dfs = []# list of dataframes

for file in os.listdir():
    try:
        df = pd.read_excel(file, sheet_name="DATA", usecols='C,D', skiprows = 1)
        df.loc[df.shape[0]] = [None, None]# to ensure that there are breaks between each recorded shower
        dfs.append(df)
    except:
        print(file)

data = pd.concat(dfs, ignore_index = True)# concatenating all data into one dataframe
data.columns = ['temp', 'label']

.DS_Store


In [3]:
start_pts = []#list where the first item is 1 for start and 0 for not start
end_pts = []#same list but for ends
for i, row in data.iterrows():
    if not pd.isnull(row['label']):
        if row['label'].startswith('Start'):
            start_pts.append([1] + data.iloc[i-4:i+5]['temp'].tolist())
        if row['label'].startswith('End'):
            end_pts.append([1] + data.iloc[i-4:i+5]['temp'].tolist())
            
initial_length = len(start_pts)
if initial_length != len(end_pts):
    print("inconsistent labels!")

random_start_indices = random.sample(range(len(data) - 9), k =  len(data) - 9)
random_end_indices = random.sample(range(len(data) - 9), k = len(data) - 9)

for i in random_start_indices:
    if not (any(pd.isnull(data.iloc[i:i+9]['temp'])) or
            (not pd.isnull(data.iloc[i+3]['label']) and data.iloc[i+3]['label'].startswith('Start'))):
        start_pts.append([0] + data.iloc[i:i+9]['temp'].tolist())
        if len(start_pts) == initial_length * 2:
            break

for i in random_end_indices:
    if not (any(pd.isnull(data.iloc[i:i+9]['temp'])) or
            (not pd.isnull(data.iloc[i+3]['label']) and data.iloc[i+3]['label'].startswith('End'))):
        end_pts.append([0] + data.iloc[i:i+9]['temp'].tolist())
        if len(end_pts) == initial_length * 2:
            break

In [4]:
start_data = tf.convert_to_tensor(start_pts)#tensors holding the label, followed by a list of 10 temperatures
start_data = tf.random.shuffle(start_data)

end_data = tf.convert_to_tensor(end_pts)
end_data = tf.random.shuffle(end_data)

2022-02-25 11:31:36.574256: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
data_length = start_data.shape[0]

start_train = start_data[:int(data_length * .7)]
start_val = start_data[int(data_length * .7):int(data_length * .85)]
start_test = start_data[int(data_length * .85):]

end_train = end_data[:int(data_length * .7)]
end_val = end_data[int(data_length * .7):int(data_length * .85)]
end_test = end_data[int(data_length * .85):]

In [29]:
start_model = Sequential(
    [
        BatchNormalization(),
        Dense(16, activation = 'relu', input_shape=(10,)),
        Dense(16, activation = 'relu'),
        Dense(1, activation = 'sigmoid')
    ]
)

start_model.compile(loss='mse', metrics=['mae'])
start_model.fit(start_train[:,1:], start_train[:,0], epochs=20)
start_val_predicted = start_model.predict(start_val[:,1:])

Epoch 1/20
3/3 [==============================] - 0s 1ms/step - loss: 0.2746 - mae: 0.5030
Epoch 2/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2635 - mae: 0.4938
Epoch 3/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2547 - mae: 0.4850
Epoch 4/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2498 - mae: 0.4833
Epoch 5/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2482 - mae: 0.4818
Epoch 6/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2401 - mae: 0.4779
Epoch 7/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2325 - mae: 0.4729
Epoch 8/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2333 - mae: 0.4717
Epoch 9/20
3/3 [==============================] - 0s 3ms/step - loss: 0.2295 - mae: 0.4690
Epoch 10/20
3/3 [==============================] - 0s 3ms/step - loss: 0.2372 - mae: 0.4763
Epoch 11/20
3/3 [==============================] - 0s 4ms/step - loss: 0.2245 - mae: 0.46

In [32]:
np.mean(np.absolute(start_val_predicted[:,0] - start_val[:,0]))

0.4998656

In [33]:
end_model = Sequential(
    [
        BatchNormalization(),
        Dense(16, activation = 'relu', input_shape=(10,)),
        Dense(16, activation = 'relu'),
        Dense(1, activation = 'sigmoid')
    ]
)

end_model.compile(loss='mse', metrics=['mae'])
end_model.fit(end_train[:,1:], end_train[:,0], epochs=20)
end_val_predicted = end_model.predict(end_val[:,1:])

Epoch 1/20
3/3 [==============================] - 0s 1ms/step - loss: 0.2677 - mae: 0.5093
Epoch 2/20
3/3 [==============================] - 0s 8ms/step - loss: 0.2614 - mae: 0.5050
Epoch 3/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2537 - mae: 0.5006
Epoch 4/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2539 - mae: 0.5003
Epoch 5/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2536 - mae: 0.4995
Epoch 6/20
3/3 [==============================] - 0s 3ms/step - loss: 0.2575 - mae: 0.5024
Epoch 7/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2525 - mae: 0.4990
Epoch 8/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2518 - mae: 0.4983
Epoch 9/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2505 - mae: 0.4973
Epoch 10/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2514 - mae: 0.4986
Epoch 11/20
3/3 [==============================] - 0s 2ms/step - loss: 0.2481 - mae: 0.49

In [34]:
np.mean(np.absolute(end_val_predicted[:,0] - end_val[:,0]))

0.36628133